In [1]:
import polars as pl

df = pl.read_parquet("./../data/car_data.parquet")

numerical_features = ["release_year", "gears", "engine_size", "cylinders"]
categorical_features = ["make", "vehicle_class", "fuel_type", "transmission_type"]
features = numerical_features + categorical_features

targets = ["emissions", "fc_mixed", "fc_city", "fc_highway"]

X = df[features]
Y = df[targets]

In [2]:
from utils import display_columns_name_mapping

df = pl.read_parquet("./../data/car_data.parquet")

df = df.rename(display_columns_name_mapping)

extra_cols_to_filter = [
    "Release year",
    "Vehicle class",
    "Fuel Type",
    "Transmission",
    "Gears",
    "Engine size (L)",
    "Cylinders",
    "CO2 emissions (g/km)",
    "Mixed (L/100 km)",
]

df.select([s for s in df if s.n_unique() > 100]).columns
df.select([pl.col(col).n_unique() > 1 for col in extra_cols_to_filter]).columns

2026-02-11 09:45:47.146 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-02-11 09:45:47.148 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


['Release year',
 'Vehicle class',
 'Fuel Type',
 'Transmission',
 'Gears',
 'Engine size (L)',
 'Cylinders',
 'CO2 emissions (g/km)',
 'Mixed (L/100 km)']

### Model definition

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        (
            "numerical",
            Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="mean")),
                    ("scaler", StandardScaler()),
                ]
            ),
            numerical_features,
        ),
    ]
)

pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("regressor", Lasso(max_iter=10000))]
)

param = {"regressor__alpha": [0.1, 1.0, 10.0, 100.0]}

In [4]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipeline, param, cv=5, n_jobs=-1, refit=True)
grid.fit(X, Y)

print(f"Best parameters : {grid.best_params_}")
print(f"Best score : {grid.best_score_:0.2f}")

Best parameters : {'regressor__alpha': 0.1}
Best score : 0.71


### Save model

In [6]:
import pickle
from pathlib import Path

filepath = Path("./../data/lasso_regression.pkl")
with filepath.open("wb") as f:
    pickle.dump(grid.best_estimator_, f)